In [155]:
import pymysql
import yaml
import types
from tqdm import tqdm
import random
import numpy as np
import torch
from psnet.config import Config
from psnet.sampling import get_idx_list
import sys
from psnet.config import  load_pkl, pkl_parser

In [137]:
with open('database.yaml', 'r') as file:
    database_info = yaml.safe_load(file)
    mysql_info = database_info['mysql']
    host = mysql_info['host']
    user = mysql_info['user']
    passwd = mysql_info['passwd']
    db=  pymysql.connect(host=host, user=user, passwd=passwd, db='edge_task_scheduling', charset='utf8')

In [138]:
arg =  types.SimpleNamespace()
arg.dataset = "alibaba_cluster_trace"
# arg.offset = 370400
arg.task_n = 277800
# arg.task_n = 10

In [141]:
def get_data_instance(self, arg):
    # query = 'SELECT MAX(id) FROM {}'.format(self.dataset)
    # self.cursor.execute(query)
    # max_id = self.cursor.fetchone()[0] - arg.task_n
    # start_id = random.randint(0, max_id)
    # 从一个随机id开始抽取数据
    query = 'SELECT * FROM {} LIMIT {}'.format(self.dataset, arg.task_n)
    self.cursor.execute(query)
    # [id, cpu,io,bandwidth,ram,timestamp,duration,latitude,longitude]
    task_list = self.cursor.fetchall()
    task_list = np.array(task_list)[:, 1:-2]
    task_list[:, 4] = abs(task_list[:, 4] - task_list[0][4])
    return torch.tensor(task_list, dtype=torch.float32).unsqueeze(0)
self = types.SimpleNamespace()
self.get_data_instance = get_data_instance

In [143]:
self.cursor =  db.cursor()
self.dataset = arg.dataset
self.file_lines = []
instance = self.get_data_instance(self,arg)
instance = instance.squeeze(0).tolist()

# print(self.file_lines[0])
print(f'Loaded "{self.dataset}" with {len(instance)} lines')

Loaded "alibaba_cluster_trace" with 277800 lines


In [156]:
print(type(instance[0][0]))
print(sys.path)
print(pkl_parser().path)

<class 'float'>
['/home/li/DIFUSCO_EDGE/difusco/co_datasets', '/home/li/miniconda3/envs/difusco_edge/lib/python37.zip', '/home/li/miniconda3/envs/difusco_edge/lib/python3.7', '/home/li/miniconda3/envs/difusco_edge/lib/python3.7/lib-dynload', '', '/home/li/miniconda3/envs/difusco_edge/lib/python3.7/site-packages', '/home/li/miniconda3/envs/difusco_edge/lib/python3.7/site-packages/IPython/extensions', '/home/li/.ipython']
./co_datasets/psnet/config/config.pkl


In [154]:
# 打开一个文件，以写入模式
task_list = []
with open("/home/li/DIF-RS/data/val/alibaba_cluster_trace_50.txt", "w+") as file:
    # 遍历列表中的每个元素
    for index,item in enumerate(instance):
        str_list = [str(i) for i in item]
        # 将每个元素写入文件，并添加换行符
        # print(type(item))
        # print(' '.join(str_list))
        file.write(' '.join(str_list) + "\n")
        task_list.append(item)
        if((index+1) % 50 == 0):
            idx_list = get_idx_list(task_list,arg.dataset)
            idx_list = [str(j) for j in idx_list]
            file.write(' '.join(idx_list) + "\n")
            task_list = []

FileNotFoundError: pkl_path

In [126]:
self.data_file = "/home/li/DIF-RS/data/train/alibaba_cluster_trace.txt"
# self.sparse_factor = sparse_factor
# self.file_lines = open(self.data_file).read().splitlines()
# print(f'Loaded "{self.data_file}" with {len(self.file_lines)} lines')
self.file_lines = []
with open(self.data_file, 'r', encoding='utf-8') as file:
        chunk = []
        for index, line in enumerate(file):
            chunk.append([float(item) for item in(line.strip().split(' '))])  # 去掉行末的换行符
            if (index + 1) % arg.task_n == 0:
                temp = chunk[0][4] # 减去第一个任务的时间，获得每个任务的相对时间
                for i,_ in enumerate(chunk):
                     chunk[i][4] = abs(chunk[i][4] - temp)
                self.file_lines.append(chunk)
                chunk = []
        # 处理最后一组不满50行的情况
        if chunk:
            self.file_lines.append(chunk)
print(self.file_lines[0])
print(f'Loaded "{self.data_file}" with {len(self.file_lines)} lines each line has {arg.task_n} task')


[[0.14000000059604645, 0.033577900379896164, 0.0042472099885344505, 0.0001284549944102764, 0.0, 22.0], [0.05000000074505806, 0.04074599966406822, 0.008879530243575573, 0.000866578018758446, 3.0, 20.0], [0.27000001072883606, 0.007546550128608942, 0.012826000340282917, 0.0009475279948674142, 4.0, 28.0], [0.27000001072883606, 0.010792099870741367, 0.01866159960627556, 0.0001379040040774271, 5.0, 30.0], [0.1599999964237213, 0.03124449960887432, 0.017170099541544914, 0.0008469360182061791, 5.0, 15.0], [0.17000000178813934, 0.028396200388669968, 0.011035400442779064, 0.0008209680090658367, 5.0, 36.0], [0.1599999964237213, 0.04817119985818863, 0.003666939912363887, 0.0005640339804813266, 6.0, 34.0], [0.28999999165534973, 0.060217298567295074, 0.004058400169014931, 0.0003572659916244447, 6.0, 13.0], [0.12999999523162842, 0.06112319976091385, 0.009291199967265129, 9.422739822184667e-05, 7.0, 10.0], [0.1599999964237213, 0.029513899236917496, 0.015450799837708473, 0.0003993390128016472, 8.0, 11.0